# Structured Data

## Key Concepts
- Claude naturally adds explanatory text and markdown formatting around structured output
- For applications needing raw data (JSON, code), this creates friction for users
- Combine assistant message prefilling + stop sequences for clean output
- Prefill with opening delimiter (e.g., "```json"), stop at closing delimiter (e.g., "```")
- Claude thinks it already started a code block and continues with just the content
- Stop sequence prevents Claude from adding closing markdown or explanations

## Important Code Patterns
- `add_assistant_message(messages, "```json")` - prefill with code block start
- `chat(messages, stop_sequences=["```"])` - stop before closing code block
- `text.strip()` - remove extra newlines from response
- `json.loads(text.strip())` - parse cleaned JSON string
- Works for any structured format: Python code, CSV, bulleted lists
- Identify what Claude wraps content in, use as prefill/stop combo

## Best Practices
- Use this technique when building apps that need copy-paste ready output
- Prefill with the opening wrapper Claude would naturally use
- Set stop sequence to the closing wrapper
- Always clean/strip responses before parsing
- Not just for JSON - applies to any structured data format
- Essential for integrating AI-generated content into applications
- Reduces user friction by eliminating manual content extraction

In [ ]:
# Install dependencies
%pip install anthropic python-dotenv

# Imports
from dotenv import load_dotenv
import os
from anthropic import Anthropic

# Load environment variables
load_dotenv()

# Create client
client = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
model = "claude-sonnet-4-0"


In [12]:
def add_user_message(messsages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)
def add_assistant_message(messsages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)
def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
         "model": model,
         "max_tokens": 1000,
         "messages": messages,
         "temperature": temperature,
         "stop_sequences": stop_sequences
    }
    if system:
         params["system"] = system
    message = client.messages.create(**params)
    return message.content[0].text

In [13]:
messages = []
add_user_message(
    messages,
    "Count from 1 to 10",
)
answer = chat(messages, stop_sequences=["5", "3, 4"])
answer


'1, 2, '

In [14]:
messages = []

add_user_message(messages, "Generate a very short event bridge rule as json")
add_assistant_message(messages, "'''json")

text = chat(messages, stop_sequences=["'''"])
text

'\n{\n  "Name": "OrderProcessingRule",\n  "EventPattern": {\n    "source": ["myapp.orders"],\n    "detail-type": ["Order Placed"]\n  },\n  "Targets": [\n    {\n      "Id": "1",\n      "Arn": "arn:aws:lambda:us-east-1:123456789012:function:ProcessOrder"\n    }\n  ],\n  "State": "ENABLED"\n}\n'

In [15]:
import json

json.loads(text.strip())

{'Name': 'OrderProcessingRule',
 'EventPattern': {'source': ['myapp.orders'], 'detail-type': ['Order Placed']},
 'Targets': [{'Id': '1',
   'Arn': 'arn:aws:lambda:us-east-1:123456789012:function:ProcessOrder'}],
 'State': 'ENABLED'}